## Exercises

For tasks 1-3, we study optimization problem
$$
\begin{align}
\min \qquad & x_1^2+x_2^2 + x_1+2x_2\\
\text{s.t.}\qquad &x_1+x_2=1\\
    &x_1,x_2\in\mathbb R
\end{align}
$$

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
#objection function: f(x) = x_1^2 + x_2^2 + x_1 + 2x_2
def f(x):
    return x[0]**2 + x[1]**2 + x[0] + 2*x[1]

#equality constrain: h(x) = x_1 + x_2 -1 ==0
def h(x):
    return x[0]+x[1]-1


1. **(2 points)** Solve the problem using the penalty function method. Note that it is not sufficient to use a fixed value for $r$.

### Solution 1
As the penalty function method, we convert our constraint problem to new function
$$
\begin{align}\
\min \qquad &Q(x) = f(x)+r\alpha(x)\\
\text{s.t.} \qquad &x\in \mathbb R^n
\end{align}
$$
f(x) is the object function;\
$\alpha(x)$ is the penalty function;
$$
\begin{align}\
\min \qquad &Q(x) = f(x) + r\sum_{i=1}^{100} h_i(x)^2 + r\sum_{j=1}^{100} ( \max(0,g_j(x))^2 )
\end{align}
$$

$$
\begin{align}\
\min \qquad &Q(x) = (x_1^2+x_2^2 + x_1+2x_2) + r(x_1+x_2-1)^2 \\
\end{align}
$$

In [3]:
#Quadratic penalty function: Q(x)= f(x) + r * sigma h_i(x)**2
def Q(f,h,r):
    return lambda x : f(x) +  r * (h(x)**2)

In [4]:
def solve_Q():
    epsilon = 10**-4
    r = 0
    x0 = np.array([float('inf')]*2)
    x1 = [0,0]
    while np.linalg.norm(x1-x0) > epsilon:
        res = minimize(Q(f,h,r), [0, 0],method='BFGS')
        x0 = x1
        x1 = np.array(res.x)
        r = r+1
        #print(x1, r)
    print(x1,r)

solve_Q()

[0.74345549 0.24345549] 96


2. **(2 points)** Solve the problem (i.e., approximate the optimal solution) using the barrier function method. Note that you need to do something a bit clever.

### Solution 2
covert the equality to inequality
$$
\begin{align}\
\qquad &g(x)=
{ 
    \begin{cases}
    h(x) + \epsilon >0 \\
    \epsilon - h(x)  <0
    \end{cases}
}\\
\qquad \text{s.t.} \qquad &x\in \mathbb R^n
\end{align}
$$

In [5]:
#barrier function method

#covert the equality to inequality
# g(x) = h(x) + epsilon > 0, epsilon-h(x) < 0
def g(x,epsilon):
    return [h(x) + epsilon, epsilon-h(x)]
# g1 = lambda x: g(x,1)
# print(g1([0,0]))
def beta(g,x):
    try:
        value = sum([ 1/max([0,ieq_j]) for ieq_j in g(x)])
        #print(value)
    except ZeroDivisionError:
        value = float('inf')
    return value

def Q_barrier(f,g,r):
    return lambda x : f(x) +  r * beta(g,x)

In [7]:
import ad
def solve_Q_barrier():
    r = 1.0
    epsilon = 1
    x0 = np.array([float('inf')]*2)
    x1 = [1,0]
    while np.linalg.norm(x1-x0) > 0.0001:
        g1 = lambda x: g(x,epsilon)
        q = Q_barrier(f,g1,r)
        res = minimize(q, x1, method='Newton-CG',
        jac=ad.gh(q)[0],hess=ad.gh(q)[1])
        x0 = x1
        x1 = res.x
        r = r/2
        epsilon = epsilon/2
    print(x1,epsilon,r)

solve_Q_barrier()


[0.74998093 0.24998093] 0.00390625 0.00390625


3. **(2 points)** Solve the problem using projected gradient method. Compare the performance to the penalty function and barrier function methods. 

In [9]:
def project_vector(A,vector):
    #convert A into a matrix
    A_matrix = np.matrix(A)
    #construct the "first row" of the matrix [[I,A^T],[A,0]]
    left_matrix_first_row = np.concatenate((np.identity(len(vector)),A_matrix.transpose()), axis=1)
    #construct the "second row" of the matrix
    left_matrix_second_row = np.concatenate((A_matrix,np.matrix(np.zeros([len(A),len(vector)+len(A)-len(A[0])]))), axis=1)
    #combine the whole matrix by combining the rows
    left_matrix = np.concatenate((left_matrix_first_row,left_matrix_second_row),axis = 0)
    #Solve the system of linear equalities from the previous page
    return np.linalg.solve(left_matrix, \
                           np.concatenate((np.matrix(vector).transpose(),\
                                           np.zeros([len(A),1])),axis=0))[:len(vector)]
# A = [[1,0,0],[0,1,0]]
# gradient = [1,1,1]
# print(project_vector(A,gradient))
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        gradient = ad.gh(f)[0](x)
        grad_proj = project_vector(A,[-i for i in gradient])#The only changes to steepest..
        grad_proj = np.array(grad_proj.transpose())[0] #... descent are here!
        x = x+grad_proj*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [10]:
res = projected_gradient_method(lambda x:f(x),[[1,1]],[1,0]\
                          ,.5,0.000001)

print(res)

(array([0.75, 0.25]), 1.875, [[0.75, 0.25], [0.75, 0.25]])


4. **(2 points)** Check the *necessary first order KKT conditions* for the solution that you found.

### KKT condition validation
$$L(x,\lambda,\mu) = f(x)- \sum_{k=1}^K\mu_kg_k(x) -\sum_{j=1}^J\lambda_jh_j(x)$$

To use the KKT condition, we need to find the gradient of the Lagrance 
$$
\begin{align}
&L(x,\lambda,\mu)  = (x_1^2+x_2^2+x_1+2x_2) + \lambda_1(x_1+x_2-1)\\
&\nabla_xL(x,\lambda,\mu) = 0\\
&\nabla_xL(x,\lambda,\mu) ={ 
    \begin{cases}
    2x_1+1-\lambda_1 = 0 \\
    2x_2+2-\lambda_1=0
    \end{cases}
}\\
&=> 2(x_1-x_2)-1 = 0
\end{align}
$$

In [2]:
def dL(x):
    return 2*(x[0]-x[1])-1
def kkt(x,h):
    if abs(dL(x)) <= h:
        return True
    else:
        return False

### Validataion
Since the x* = [0.75,0.25] is the optimum point, so we will check at points from[0.0,0.0] to [1.0,1.0]. with step 0.01, tolerence 0.01

In [30]:
# h = 0.01
# i = np.linspace(0,1,101)
# j = np.linspace(0,1,101)
# for ik in i:
#     for jk in j:
#         x0 = [ik,jk]
#         if kkt(x0,h):
#             print("True validated at point:", x0)


True validated at point: [0.5, 0.0]
True validated at point: [0.51, 0.01]
True validated at point: [0.52, 0.02]
True validated at point: [0.53, 0.03]
True validated at point: [0.54, 0.04]
True validated at point: [0.55, 0.05]
True validated at point: [0.56, 0.06]
True validated at point: [0.5700000000000001, 0.07]
True validated at point: [0.58, 0.08]
True validated at point: [0.59, 0.09]
True validated at point: [0.6, 0.1]
True validated at point: [0.61, 0.11]
True validated at point: [0.62, 0.12]
True validated at point: [0.63, 0.13]
True validated at point: [0.64, 0.14]
True validated at point: [0.65, 0.15]
True validated at point: [0.66, 0.16]
True validated at point: [0.67, 0.17]
True validated at point: [0.68, 0.18]
True validated at point: [0.6900000000000001, 0.19]
True validated at point: [0.7000000000000001, 0.2]
True validated at point: [0.71, 0.21]
True validated at point: [0.72, 0.22]
True validated at point: [0.73, 0.23]
True validated at point: [0.74, 0.24]
True validate

In [4]:
x = [0.75,0.25]
if kkt(x,0.00001):
    print("True validated at point", x)

True validated at point [0.75, 0.25]
